### Swarm set up

In [1]:
from swarm import Swarm, Agent

client = Swarm()

### Basic use

In [2]:
agent = Agent(
    name="Agent",
    instructions="You are a helpful agent.",
)

In [3]:
messages = [{"role": "user", "content": "What is the capital of Portugal"}]
response = client.run(agent=agent, messages=messages)

print(response.messages[-1]["content"])

The capital of Portugal is Lisbon.


### Handoff to another agent

In [4]:
english_agent = Agent(
    name="English Agent",
    instructions="You only speak English.",
)

spanish_agent = Agent(
    name="Spanish Agent",
    instructions="You only speak Spanish.",
)


def transfer_to_spanish_agent():
    """Transfer spanish speaking users immediately."""
    return spanish_agent


english_agent.functions.append(transfer_to_spanish_agent)



In [5]:
messages = [{"role": "user", "content": "Hola. ¿Como estás?"}]
response = client.run(agent=english_agent, messages=messages)

print(response.messages[-1]["content"])

¡Hola! Estoy bien, gracias. ¿Y tú, cómo estás?


In [6]:
messages = [{"role": "user", "content": "Hi. How are you?"}]
response = client.run(agent=english_agent, messages=messages)

print(response.messages[-1]["content"])

I'm doing well, thank you! How can I assist you today?


### Add a tool

In [20]:
import wikipedia

def wikipedia_lookup(q: str) -> str:
    """Look up a query in Wikipedia and return the result"""
    try: return wikipedia.page(q).summary
    except: return None

def wikipedia_search(q: str) -> str:
    """Search for a topic in Wikipedia and return the result"""
    return wikipedia.search(q)

In [157]:
topics = wikipedia.search('EU capitals')
topics

['European Capital of Culture',
 'List of national capitals',
 'Religion in the European Union',
 'European Union',
 'Vienna Capitals',
 'European Council on Foreign Relations',
 'Crime in London',
 'Crime in Bucharest',
 'Flag of Europe',
 'Ramona Strugariu']

In [161]:
entry = wikipedia_lookup(topics[3])
print(entry)

The European Union (EU) is a supranational political and economic union of 27 member states that are located primarily in Europe. The Union has a total area of 4,233,255 km2 (1,634,469 sq mi) and an estimated total population of over 449 million. The EU has often been described as a sui generis political entity combining the characteristics of both a federation and a confederation.
Containing 5.8% of the world population in 2020, EU member states generated a nominal gross domestic product (GDP) of around US$16.6 trillion in 2022, constituting approximately one sixth of global nominal GDP. Additionally, all EU states except Bulgaria have a very high Human Development Index according to the United Nations Development Programme. Its cornerstone, the Customs Union, paved the way to establishing an internal single market based on standardised legal framework and legislation that applies in all member states in those matters, and only those matters, where the states have agreed to act as one

### A tool-using agent 

In [106]:
wikiagent = Agent(
    name="Agent",
    instructions="""You are a helpful agent that answers user queries by finding and analysing information from Wikipedia.
                        You should follow the following sequence:
                        1. Work out what the user is interested in.
                        2. Pick keywords
                        3. Use the lookup tool with the most relevant keyword
                        4. From the resulting list of results pick the most relvant to the user query and search for it using the search tool 
                        5. If you are able provided an answer from that information stop and answer,
                        otherwise, start again from step 2 but with different keywords. 
                        """,
    functions=[wikipedia_lookup, wikipedia_search],
)

In [107]:
messages = [{"role": "user", "content": "What is the population of the largest city in Scotland"}]

response = client.run(agent=wikiagent, messages=messages)
print(response.messages[-1]["content"])

Glasgow is the largest city in Scotland by population, but the specific population number isn't provided in the summary. You might need to check the most recent data from official sources such as the National Records of Scotland for the exact number.


In [108]:
for m in response.messages:
    try: print(f"Tool call: {m['tool_calls']}")
    except: pass
    try: print(f"Content: {m['content']}")
    except: pass

Tool call: [{'id': 'call_DkOUR7CXi3Zr3nLvEESaG96N', 'function': {'arguments': '{"q":"Largest city in Scotland"}', 'name': 'wikipedia_lookup'}, 'type': 'function'}]
Content: None
Content: This list of towns and cities in Scotland with a population of more than 15,000 is ordered by population, as defined and compiled by the National Records of Scotland organisation. Glasgow is the largest city in Scotland by population, whilst the capital city, Edinburgh, is the second largest by population and largest by area (although the Aberdeen and Edinburgh local authority areas contain far more undeveloped land and have a lower population density than the council areas of Dundee and Glasgow; these are the only four city-districts in the country). The city of Stirling has the smallest population amongst Scotland's cities, with an estimated population of just over 37,000 inhabitants. In total, Scotland consists of eight cities, with multiple larger towns, the largest town being Paisley. 
The section

### A simple workflow with one agent handing of to another

In [153]:
def transfer_to_pr_agent():
    return pr_agent

agent = Agent(
    name="Agent",
    instructions="""You are a helpful agent that answers user queries by finding and analysing information from Wikipedia.
                    You will be given a city or other location and you must retrieve it's entry on Wikipedia and then hand over to the PR Agent.""",
    functions=[wikipedia_lookup, transfer_to_pr_agent],
)

pr_agent = Agent(
    name="PR Agent",
    instructions="""You are an experienced PR copywriter. 
                    Use only the information retrieved from Wikipedia to write an enthusiastic 100-word summary of the topic 
                    that would be suitable for a promonional campaign.
                    Explain how you used the original material to create the result""",
)





In [154]:
messages = [{"role": "user", "content": "Paris"}]

response = client.run(agent=agent, messages=messages)
print(response.messages[-1]["content"])

Welcome to Paris, the breathtaking capital of France, where romance dances in the air and culture knows no bounds! As the heart of finance, fashion, and gastronomy since the 17th century, Paris shines as the iconic \"City of Light,\" home to 2.1 million welcoming residents and a treasury of art and history. With world-renowned landmarks like the Louvre, the majestic Seine, and a vibrant arts scene, Paris captivates 12 million visitors in its embrace. Experience seamless connections through thriving transport networks, indulge in epicurean delights, and feel the pulse of creativity and innovation in this unparalleled metropolis. Paris awaits your discovery!

---

I crafted this exciting summary by highlighting key aspects from the Wikipedia entry on Paris. The content emphasized the city's historic importance as a cultural, financial, and gastronomical hub while accentuating its renowned symbols like the Louvre and the Seine. The population and importance of its transport system, alongs

In [155]:
for m in response.messages:
    try: print(f"Tool call: {m['tool_calls']}")
    except: pass
    try: print(f"Content: {m['content']}")
    except: pass

Tool call: [{'id': 'call_5KAWDJeA6vDx0gwPPyFPM0et', 'function': {'arguments': '{"q":"Paris"}', 'name': 'wikipedia_lookup'}, 'type': 'function'}]
Content: None
Content: Paris (French pronunciation: [paʁi] ) is the capital and largest city of France. With an official estimated population of 2,102,650 residents in January 2023 in an area of more than 105 km2 (41 sq mi), Paris is the fourth-largest city in the European Union and the 30th most densely populated city in the world in 2022. Since the 17th century, Paris has been one of the world's major centres of finance, diplomacy, commerce, culture, fashion, and gastronomy. For its leading role in the arts and sciences, as well as its early and extensive system of street lighting, in the 19th century, it became known as the City of Light.
The City of Paris is the centre of the Île-de-France region, or Paris Region, with an official estimated population of 12,271,794 inhabitants in January 2023, or about 19% of the population of France. The 

In [156]:
for m in response.messages:
    print(m)

{'content': None, 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'call_5KAWDJeA6vDx0gwPPyFPM0et', 'function': {'arguments': '{"q":"Paris"}', 'name': 'wikipedia_lookup'}, 'type': 'function'}], 'sender': 'Agent'}
{'role': 'tool', 'tool_call_id': 'call_5KAWDJeA6vDx0gwPPyFPM0et', 'tool_name': 'wikipedia_lookup', 'content': "Paris (French pronunciation: [paʁi] ) is the capital and largest city of France. With an official estimated population of 2,102,650 residents in January 2023 in an area of more than 105 km2 (41 sq mi), Paris is the fourth-largest city in the European Union and the 30th most densely populated city in the world in 2022. Since the 17th century, Paris has been one of the world's major centres of finance, diplomacy, commerce, culture, fashion, and gastronomy. For its leading role in the arts and sciences, as well as its early and extensive system of street lighting, in the 19th century, it became known as the City of Light.\